In [1]:
import torch


def BasicConv2d(dim_in, dim_out, kernel_size, stride, padding):
    return torch.nn.Sequential(
        torch.nn.Conv2d(dim_in,
                        dim_out,
                        kernel_size=kernel_size,
                        stride=stride,
                        padding=padding,
                        bias=False),
        torch.nn.BatchNorm2d(dim_out, eps=0.001, momentum=0.1, affine=True),
        torch.nn.ReLU(inplace=False),
    )


BasicConv2d(3, 5, 3, 1, 1)(torch.randn(2, 3, 15, 15)).shape

torch.Size([2, 5, 15, 15])

In [2]:
class Block35(torch.nn.Module):

    def __init__(self):
        super().__init__()

        self.s0 = BasicConv2d(256, 32, kernel_size=1, stride=1, padding=0)

        self.s1 = torch.nn.Sequential(
            BasicConv2d(256, 32, kernel_size=1, stride=1, padding=0),
            BasicConv2d(32, 32, kernel_size=3, stride=1, padding=1),
        )

        self.s2 = torch.nn.Sequential(
            BasicConv2d(256, 32, kernel_size=1, stride=1, padding=0),
            BasicConv2d(32, 32, kernel_size=3, stride=1, padding=1),
            BasicConv2d(32, 32, kernel_size=3, stride=1, padding=1),
        )

        self.cnn = torch.nn.Conv2d(96, 256, kernel_size=1, stride=1, padding=0)
        self.act = torch.nn.ReLU(inplace=False)

    def forward(self, x):
        out = torch.cat((self.s0(x), self.s1(x), self.s2(x)), 1)
        out = self.cnn(out)
        out = out * 0.17 + x
        out = self.act(out)
        return out


Block35()(torch.randn(2, 256, 15, 15)).shape

torch.Size([2, 256, 15, 15])

In [3]:
class Block17(torch.nn.Module):

    def __init__(self):
        super().__init__()

        self.s0 = BasicConv2d(896, 128, kernel_size=1, stride=1, padding=0)

        self.s1 = torch.nn.Sequential(
            BasicConv2d(896, 128, kernel_size=1, stride=1, padding=0),
            BasicConv2d(128, 128, kernel_size=(1, 7), stride=1,
                        padding=(0, 3)),
            BasicConv2d(128, 128, kernel_size=(7, 1), stride=1,
                        padding=(3, 0)),
        )

        self.cnn = torch.nn.Conv2d(256,
                                   896,
                                   kernel_size=1,
                                   stride=1,
                                   padding=0)
        self.act = torch.nn.ReLU(inplace=False)

    def forward(self, x):
        out = torch.cat((self.s0(x), self.s1(x)), 1)
        out = self.cnn(out)
        out = out * 0.1 + x
        out = self.act(out)
        return out


Block17()(torch.randn(2, 896, 15, 15)).shape

torch.Size([2, 896, 15, 15])

In [4]:
class Mixed_6a(torch.nn.Module):

    def __init__(self):
        super().__init__()

        self.s0 = BasicConv2d(256, 384, kernel_size=3, stride=2, padding=0)

        self.s1 = torch.nn.Sequential(
            BasicConv2d(256, 192, kernel_size=1, stride=1, padding=0),
            BasicConv2d(192, 192, kernel_size=3, stride=1, padding=1),
            BasicConv2d(192, 256, kernel_size=3, stride=2, padding=0),
        )

        self.s2 = torch.nn.MaxPool2d(3, stride=2, padding=0)

    def forward(self, x):
        out = torch.cat((self.s0(x), self.s1(x), self.s2(x)), 1)

        return out


Mixed_6a()(torch.randn(2, 256, 15, 15)).shape

torch.Size([2, 896, 7, 7])

In [5]:
class Mixed_7a(torch.nn.Module):

    def __init__(self):
        super().__init__()

        self.s0 = torch.nn.Sequential(
            BasicConv2d(896, 256, kernel_size=1, stride=1, padding=0),
            BasicConv2d(256, 384, kernel_size=3, stride=2, padding=0))

        self.s1 = torch.nn.Sequential(
            BasicConv2d(896, 256, kernel_size=1, stride=1, padding=0),
            BasicConv2d(256, 256, kernel_size=3, stride=2, padding=0))

        self.s2 = torch.nn.Sequential(
            BasicConv2d(896, 256, kernel_size=1, stride=1, padding=0),
            BasicConv2d(256, 256, kernel_size=3, stride=1, padding=1),
            BasicConv2d(256, 256, kernel_size=3, stride=2, padding=0))

        self.s3 = torch.nn.MaxPool2d(3, stride=2)

    def forward(self, x):
        out = torch.cat((self.s0(x), self.s1(x), self.s2(x), self.s3(x)), 1)

        return out


Mixed_7a()(torch.randn(2, 896, 15, 15)).shape

torch.Size([2, 1792, 7, 7])

In [6]:
class Block8(torch.nn.Module):

    def __init__(self, scale, act):
        super().__init__()

        self.scale = scale
        self.act = act

        self.s0 = BasicConv2d(1792, 192, kernel_size=1, stride=1, padding=0)

        self.s1 = torch.nn.Sequential(
            BasicConv2d(1792, 192, kernel_size=1, stride=1, padding=0),
            BasicConv2d(192, 192, kernel_size=(1, 3), stride=1,
                        padding=(0, 1)),
            BasicConv2d(192, 192, kernel_size=(3, 1), stride=1,
                        padding=(1, 0)))

        self.cnn = torch.nn.Conv2d(384, 1792, kernel_size=1, stride=1)
        if self.act:
            self.act = torch.nn.ReLU(inplace=False)

    def forward(self, x):
        out0 = self.s0(x)
        out1 = self.s1(x)
        out = torch.cat((out0, out1), 1)
        out = self.cnn(out)
        out = out * self.scale + x
        if self.act:
            out = self.act(out)
        return out


Block8(1.0, True)(torch.randn(2, 1792, 15, 15)).shape

torch.Size([2, 1792, 15, 15])

In [7]:
model = torch.nn.Sequential(
    BasicConv2d(3, 32, kernel_size=3, stride=2, padding=0),
    BasicConv2d(32, 32, kernel_size=3, stride=1, padding=0),
    BasicConv2d(32, 64, kernel_size=3, stride=1, padding=1),
    torch.nn.MaxPool2d(3, stride=2),
    BasicConv2d(64, 80, kernel_size=1, stride=1, padding=0),
    BasicConv2d(80, 192, kernel_size=3, stride=1, padding=0),
    BasicConv2d(192, 256, kernel_size=3, stride=2, padding=0),
    Block35(),
    Block35(),
    Block35(),
    Block35(),
    Block35(),
    Mixed_6a(),
    Block17(),
    Block17(),
    Block17(),
    Block17(),
    Block17(),
    Block17(),
    Block17(),
    Block17(),
    Block17(),
    Block17(),
    Mixed_7a(),
    Block8(scale=0.2, act=True),
    Block8(scale=0.2, act=True),
    Block8(scale=0.2, act=True),
    Block8(scale=0.2, act=True),
    Block8(scale=0.2, act=True),
    Block8(scale=1.0, act=False),
    torch.nn.AdaptiveAvgPool2d(1),
    torch.nn.Dropout(0.6),
    torch.nn.Flatten(),
    torch.nn.Linear(1792, 512, bias=False),
    torch.nn.BatchNorm1d(512, eps=0.001, momentum=0.1, affine=True),
)

model(torch.randn(2, 3, 160, 160)).shape

torch.Size([2, 512])

In [8]:
# params = torch.load('data/20180402-114759-vggface2.pt')


# def load_block(s, prefix):
#     s[0].weight.data = params['%s.conv.weight' % prefix]
#     s[1].weight.data = params['%s.bn.weight' % prefix]
#     s[1].bias.data = params['%s.bn.bias' % prefix]
#     s[1].running_mean.bias = params['%s.bn.running_mean' % prefix]
#     s[1].running_var.bias = params['%s.bn.running_var' % prefix]
#     s[1].num_batches_tracked.bias = params['%s.bn.num_batches_tracked' %
#                                            prefix]


# def load_cnn(cnn, prefix):
#     cnn.weight.data = params['%s.weight' % prefix]
#     cnn.bias.data = params['%s.bias' % prefix]


# load_block(model[0], 'conv2d_1a')
# load_block(model[1], 'conv2d_2a')
# load_block(model[2], 'conv2d_2b')
# load_block(model[4], 'conv2d_3b')
# load_block(model[5], 'conv2d_4a')
# load_block(model[6], 'conv2d_4b')

# for i in range(5):
#     load_block(model[7 + i].s0, 'repeat_1.%d.branch0' % i)
#     load_block(model[7 + i].s1[0], 'repeat_1.%d.branch1.0' % i)
#     load_block(model[7 + i].s1[1], 'repeat_1.%d.branch1.1' % i)
#     load_block(model[7 + i].s2[0], 'repeat_1.%d.branch2.0' % i)
#     load_block(model[7 + i].s2[1], 'repeat_1.%d.branch2.1' % i)
#     load_block(model[7 + i].s2[2], 'repeat_1.%d.branch2.2' % i)
#     load_cnn(model[7 + i].cnn, 'repeat_1.%d.conv2d' % i)

# load_block(model[12].s0, 'mixed_6a.branch0')
# load_block(model[12].s1[0], 'mixed_6a.branch1.0')
# load_block(model[12].s1[1], 'mixed_6a.branch1.1')
# load_block(model[12].s1[2], 'mixed_6a.branch1.2')

# for i in range(10):
#     load_block(model[13 + i].s0, 'repeat_2.%d.branch0' % i)
#     load_block(model[13 + i].s1[0], 'repeat_2.%d.branch1.0' % i)
#     load_block(model[13 + i].s1[1], 'repeat_2.%d.branch1.1' % i)
#     load_block(model[13 + i].s1[2], 'repeat_2.%d.branch1.2' % i)
#     load_cnn(model[13 + i].cnn, 'repeat_2.%d.conv2d' % i)

# load_block(model[23].s0[0], 'mixed_7a.branch0.0')
# load_block(model[23].s0[1], 'mixed_7a.branch0.1')
# load_block(model[23].s1[0], 'mixed_7a.branch1.0')
# load_block(model[23].s1[1], 'mixed_7a.branch1.1')
# load_block(model[23].s2[0], 'mixed_7a.branch2.0')
# load_block(model[23].s2[1], 'mixed_7a.branch2.1')
# load_block(model[23].s2[2], 'mixed_7a.branch2.2')

# for i in range(5):
#     load_block(model[24 + i].s0, 'repeat_3.%d.branch0' % i)
#     load_block(model[24 + i].s1[0], 'repeat_3.%d.branch1.0' % i)
#     load_block(model[24 + i].s1[1], 'repeat_3.%d.branch1.1' % i)
#     load_block(model[24 + i].s1[2], 'repeat_3.%d.branch1.2' % i)
#     load_cnn(model[24 + i].cnn, 'repeat_3.%d.conv2d' % i)

# load_block(model[29].s0, 'block8.branch0')
# load_block(model[29].s1[0], 'block8.branch1.0')
# load_block(model[29].s1[1], 'block8.branch1.1')
# load_block(model[29].s1[2], 'block8.branch1.2')
# load_cnn(model[29].cnn, 'block8.conv2d')

# model[33].weight.data = params['last_linear.weight']
# model[34].weight.data = params['last_bn.weight']
# model[34].bias.data = params['last_bn.bias']
# model[34].running_mean.data = params['last_bn.running_mean']
# model[34].running_var.data = params['last_bn.running_var']
# model[34].num_batches_tracked.data = params['last_bn.num_batches_tracked']

In [9]:
from transformers import PreTrainedModel, PretrainedConfig


#包装类
class Facenet(PreTrainedModel):
    config_class = PretrainedConfig

    def __init__(self, config):
        super().__init__(config)
        self.s = torch.nn.Sequential(
            model,
            torch.nn.Linear(512, 105),
        )

    def forward(self, x):
        return self.s(x)


# Facenet(PretrainedConfig()).push_to_hub(
#     repo_id='lansinuote/simple_facenet',
#     token=open('/root/hub_token.txt').read().strip())

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/94.5M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/lansinuote/simple_facenet/commit/d5fb3a858bf539f4e5fc7d35d81ecb8f8eab2c18', commit_message='Upload Facenet', commit_description='', oid='d5fb3a858bf539f4e5fc7d35d81ecb8f8eab2c18', pr_url=None, pr_revision=None, pr_num=None)